In [46]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

In [47]:
import warnings
warnings.filterwarnings('ignore')

In [48]:
data = pd.read_csv('data_split')

In [49]:
data

,Unnamed: 0,Variable_1,Variable_2,Variable_3,Variable_4,Variable_6,Variable_7,Variable_8,Variable_9,Variable_10,...,Variable_35,Variable_36,Variable_37,Variable_39,Variable_40,Variable_41,Target,Variable_5,Variable_14,Variable_45
0,0,35.0,1.0,1.0,1.0,72.0,27.0,100.0,190.0,100.0,...,0.0,0.0,17550.5,699.0,0.0,0.0,1.0,0,11.0,1
1,1,30.0,1.0,1.0,1.0,25.0,26.0,199.0,199.0,199.0,...,0.0,0.0,40147.0,100.0,0.0,0.0,1.0,0,11.0,2
2,2,30.0,1.0,1.0,1.0,65.0,21.0,100.0,100.0,100.0,...,0.0,0.0,499.0,300.0,0.0,0.0,0.0,0,6.0,1
3,3,30.0,1.0,1.0,1.0,65.0,48.0,199.0,199.0,199.0,...,0.0,0.0,17550.5,699.0,0.0,0.0,1.0,0,3.0,1
4,4,30.0,2.0,1.0,1.0,4.0,28.0,199.0,600.0,199.0,...,0.0,0.0,17550.5,699.0,0.0,0.0,1.0,0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2052,2052,30.0,1.0,1.0,1.0,54.0,23.0,199.0,120.0,120.0,...,0.0,0.0,1083.0,699.0,0.0,0.0,1.0,0,5.0,2
2053,2053,30.0,2.0,1.0,1.0,66.0,46.0,300.0,199.0,199.0,...,0.0,0.0,24566.0,699.0,1.0,1.0,1.0,0,6.0,1
2054,2054,30.0,1.0,1.0,1.0,22.0,20.0,199.0,199.0,199.0,...,0.0,0.0,278.0,699.0,0.0,0.0,1.0,0,6.0,1
2055,2055,60.0,1.0,1.0,1.0,76.0,47.0,100.0,500.0,300.0,...,0.0,0.0,17550.5,699.0,0.0,0.0,1.0,0,6.0,1


In [50]:
X=data.drop('Target', axis=1)
y=data['Target']

In [51]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [52]:
bestfeatures = SelectKBest(score_func=chi2, k=15)
fit = bestfeatures.fit(X,y)

In [53]:
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

In [54]:
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']

In [55]:
a=(featureScores.nlargest(13,'Score'))

In [56]:
b=[i for i in a['Specs']]

In [57]:
X1=X[b]

In [58]:
from sklearn.preprocessing import StandardScaler
sca= StandardScaler()
X2 = sca.fit_transform(X1)

In [59]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X2, y, test_size=0.15, random_state=True)

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import GridSearchCV

In [61]:
log_class=LogisticRegression()
grid={'C':10.0 **np.arange(-2,3),'penalty':['l1','l2']}
cv=KFold(n_splits=5,random_state=None,shuffle=False)

In [62]:
clf=GridSearchCV(log_class,grid,cv=cv,n_jobs=-1,scoring='f1_macro')
clf.fit(X_train,y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro')

In [63]:
y_pred=clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[  0  89]
 [  0 220]]
0.7119741100323624
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        89
         1.0       0.71      1.00      0.83       220

    accuracy                           0.71       309
   macro avg       0.36      0.50      0.42       309
weighted avg       0.51      0.71      0.59       309



In [100]:
class_weight=dict({0:140,1:68})

In [118]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier(class_weight=class_weight,n_estimators=100, max_depth=5)
classifier.fit(X_train,y_train)

RandomForestClassifier(class_weight={0: 140, 1: 68}, max_depth=5)

In [119]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[ 59  30]
 [ 89 131]]
0.6148867313915858
              precision    recall  f1-score   support

         0.0       0.40      0.66      0.50        89
         1.0       0.81      0.60      0.69       220

    accuracy                           0.61       309
   macro avg       0.61      0.63      0.59       309
weighted avg       0.69      0.61      0.63       309



In [124]:
model = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=class_weight, colsample_bytree=0.9,
               importance_type='split', learning_rate=0.15, max_depth=3,
               min_child_samples=30, min_child_weight=22, min_split_gain=0.2,
               n_estimators=50, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.1, reg_lambda=0.1, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
model.fit(X_train,y_train,eval_set=[(X_test,y_test),(X_train,y_train)],
          verbose=30,eval_metric='logloss')

[30]	training's binary_logloss: 0.604797	valid_0's binary_logloss: 0.60446


LGBMClassifier(class_weight={0: 140, 1: 68}, colsample_bytree=0.9,
               learning_rate=0.15, max_depth=3, min_child_samples=30,
               min_child_weight=22, min_split_gain=0.2, n_estimators=50,
               reg_alpha=0.1, reg_lambda=0.1, silent=True)

In [125]:
print('Training accuracy {:.4f}'.format(model.score(X_train,y_train)))
print('Testing accuracy {:.4f}'.format(model.score(X_test,y_test)))

Training accuracy 0.6693
Testing accuracy 0.6181


In [128]:
y2=model.predict_proba(X_test)[:,1]
s2=roc_auc_score(y_test,y2)

In [129]:
s2

0.7039070480081717

In [130]:
y_pr=model.predict(X_test)

In [131]:
print(confusion_matrix(y_test, y_pr))
print(accuracy_score(y_test,y_pr))
print(classification_report(y_test,y_pr))

[[ 55  34]
 [ 84 136]]
0.6181229773462783
              precision    recall  f1-score   support

         0.0       0.40      0.62      0.48        89
         1.0       0.80      0.62      0.70       220

    accuracy                           0.62       309
   macro avg       0.60      0.62      0.59       309
weighted avg       0.68      0.62      0.64       309



In [120]:
s1=roc_auc_score(y_test,y_pred)
s1

0.6291879468845761

In [121]:
from collections import Counter
Counter(y_train)

Counter({1.0: 1204, 0.0: 544})

from collections import Counter
from imblearn.under_sampling import NearMiss
ns=NearMiss(0.8)
X_train_ns,y_train_ns=ns.fit_resample(X_train,y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_ns)))

In [68]:
from imblearn.under_sampling import RandomUnderSampler  

under_sampler = RandomUnderSampler()
X_res, y_res = under_sampler.fit_resample(X_train,y_train)

In [69]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_res,y_res)

RandomForestClassifier()

In [70]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[ 64  25]
 [ 98 122]]
0.6019417475728155
              precision    recall  f1-score   support

         0.0       0.40      0.72      0.51        89
         1.0       0.83      0.55      0.66       220

    accuracy                           0.60       309
   macro avg       0.61      0.64      0.59       309
weighted avg       0.70      0.60      0.62       309



# Over Sampling

In [71]:
from imblearn.over_sampling import RandomOverSampler

In [72]:
os=RandomOverSampler(0.75)
X_train_ns,y_train_ns=os.fit_resample(X_train,y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_ns)))

The number of classes before fit Counter({1.0: 1204, 0.0: 544})
The number of classes after fit Counter({1.0: 1204, 0.0: 903})


In [73]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train_ns,y_train_ns)

RandomForestClassifier()

In [74]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[ 35  54]
 [ 41 179]]
0.6925566343042071
              precision    recall  f1-score   support

         0.0       0.46      0.39      0.42        89
         1.0       0.77      0.81      0.79       220

    accuracy                           0.69       309
   macro avg       0.61      0.60      0.61       309
weighted avg       0.68      0.69      0.68       309



In [75]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [76]:
y_prob=classifier.predict_proba(X_test)[:,1]

In [77]:
s=roc_auc_score(y_test,y_prob)

In [78]:
s

0.6683861082737487

In [79]:
from imblearn.combine import SMOTETomek

In [81]:
os=SMOTETomek(0.75)
X_train_ns,y_train_ns=os.fit_resample(X_train,y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_ns)))

The number of classes before fit Counter({1.0: 1204, 0.0: 544})
The number of classes after fit Counter({1.0: 1111, 0.0: 810})


In [82]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train_ns,y_train_ns)

RandomForestClassifier()

In [88]:
import lightgbm as lgb

In [91]:
model = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.15, max_depth=3,
               min_child_samples=20, min_child_weight=1, min_split_gain=0.0,
               n_estimators=40, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
model.fit(X_train,y_train,eval_set=[(X_test,y_test),(X_res,y_res)],
          verbose=20,eval_metric='logloss')

[20]	valid_0's binary_logloss: 0.547054	valid_1's binary_logloss: 0.684007
[40]	valid_0's binary_logloss: 0.542073	valid_1's binary_logloss: 0.6572


LGBMClassifier(learning_rate=0.15, max_depth=3, min_child_weight=1,
               n_estimators=40, silent=True)

In [93]:
print('Training accuracy {:.4f}'.format(model.score(X_res,y_res)))
print('Testing accuracy {:.4f}'.format(model.score(X_test,y_test)))

Training accuracy 0.5294
Testing accuracy 0.7249


In [83]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[ 34  55]
 [ 43 177]]
0.6828478964401294
              precision    recall  f1-score   support

         0.0       0.44      0.38      0.41        89
         1.0       0.76      0.80      0.78       220

    accuracy                           0.68       309
   macro avg       0.60      0.59      0.60       309
weighted avg       0.67      0.68      0.68       309



In [84]:
from imblearn.ensemble import EasyEnsembleClassifier

In [86]:
easy=EasyEnsembleClassifier()
easy.fit(X_train,y_train)

EasyEnsembleClassifier()

In [87]:
y_pred=easy.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[ 62  27]
 [112 108]]
0.5501618122977346
              precision    recall  f1-score   support

         0.0       0.36      0.70      0.47        89
         1.0       0.80      0.49      0.61       220

    accuracy                           0.55       309
   macro avg       0.58      0.59      0.54       309
weighted avg       0.67      0.55      0.57       309

